<a href="https://colab.research.google.com/github/wujj0326/business_agglomeration/blob/master/Data/Yelp_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yelp Dataset for RMDS Capstone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
import json 
import tarfile
from pandas import json_normalize
import pandas as pd
import random
import numpy as np


In [4]:
# Unzip the tarfile
with tarfile.open("/content/drive/My Drive/Capstone/yelp_dataset.tar") as tf:
  members = tf.getmembers()
  b = tf.extractfile(members[5])
  r = tf.extractfile(members[6])
  b1 = b.readlines()
  r1 = r.readlines()

In [5]:
# read as json and filter it to include only Las Vegas
b_json = []
b_vegas = []
for st in b1:
  b_json.append(json.loads(st))
for b_filter in b_json:
  if b_filter['city']=='Las Vegas':
    b_vegas.append(b_filter)

In [ ]:
# the length of business data
len(b_json)

209393

In [ ]:
# The number of business in Las Vegas
len(b_vegas)

31631

In [6]:
# The business ID for the business in Las Vegas
b_id = []
for business in b_vegas:
  b_id.append(business['business_id'])

In [7]:
# read the review data as json
r_json = []
for st in r1:
  r_json.append(json.loads(st))

In [7]:
# The length of review data
len(r_json)

8021122

In [8]:
# the business id in review data
review_freq = []
for i in r_json:
  review_freq.append(i['business_id'])

In [9]:
# count the freq of each id
from collections import Counter
z = Counter(review_freq)

In [ ]:
# the length of unique id
len(z)

209393

In [10]:
# the ids that have more than 50 reviews
bid_more_than_50 = [k for k,v in z.items() if v>50]

In [11]:
# the length of ids from previous cell
len(bid_more_than_50)

32420

In [ ]:
# extract the ids that are in both bid more than 50 and bid in vegas
b50_vegas = []
for b50, b_veg in zip(bid_more_than_50, b_id):
  if b50 in b_id and b_veg in bid_more_than_50:
    b50_vegas.append(b50)

In [16]:
len(b50_vegas)

1873

In [17]:
# extract the review that are in b50_vegas
r_50 = []
for r_filter in r_json:
  if r_filter['business_id'] in b50_vegas:
    r_50.append(r_filter)

In [18]:
len(r_50)

525106

In [19]:
review_50 = json_normalize(r_50)

In [20]:
business_vegas = json_normalize(b_vegas)

In [21]:
yelp_vegas = pd.merge(review_50, business_vegas, how='inner', left_on='business_id', right_on='business_id')

In [25]:
# To check if there are ids that include only 1 review
a = np.where(yelp_vegas['business_id'].value_counts()==1)
len(a[0])

0

In [29]:
yelp_vegas.shape

(525106, 68)

In [28]:
yelp_vegas.columns

Index(['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars_y', 'review_count',
       'is_open', 'categories', 'attributes.BusinessParking',
       'attributes.BikeParking', 'attributes.WiFi',
       'attributes.RestaurantsPriceRange2',
       'attributes.BusinessAcceptsCreditCards', 'hours.Monday',
       'hours.Tuesday', 'hours.Wednesday', 'hours.Thursday', 'hours.Friday',
       'hours.Saturday', 'hours', 'attributes.Alcohol',
       'attributes.GoodForKids', 'attributes.RestaurantsDelivery',
       'attributes.RestaurantsReservations', 'attributes.OutdoorSeating',
       'attributes.RestaurantsGoodForGroups', 'attributes.RestaurantsTakeOut',
       'attributes.Caters', 'attributes.HasTV', 'attributes.Ambience',
       'attributes.RestaurantsAttire', 'attributes.NoiseLevel', 'hours.Sunday',
       'attributes.ByAppointmentOnly', 'attributes',
    

In [34]:
yelp_vegas_50 = yelp_vegas.loc[:,['review_id', 'user_id', 'business_id', 'stars_x', 'useful', 'funny',
       'cool', 'text', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude','categories']]

In [35]:
yelp_vegas_50['business_id'].value_counts()

4JNXUYY8wbaaDmk3BPzlWw    9536
K7lWdNUhCbcnEvI0NhGewg    7594
cYwJA2A6I12KNkm2rtXd5g    5586
DkYS3arLOhA8si5uUEmHOw    5370
2weQS-RnoOBhb1KsHKyoSQ    4953
                          ... 
frBAaI0FRj1BwnToT1AFZA      51
ekYwEVTOF2_h0wUxgea4Xw      51
NdgsBrZKSsmCNBDfj_0L0A      51
JXvLE5CRN88sccZtNeMIrQ      51
oXL7Jm7N_dUMKV4Oe-C1lA      51
Name: business_id, Length: 1873, dtype: int64

In [36]:
yelp_vegas_50.to_csv('/content/drive/My Drive/Capstone/yelp_vegas_50.csv',index=False)